We use echo-evolution data with a two-qubit noise level $q_{2} = 0.01$ and 5 points in time from 0 to $\pi$. The data are deployed in time in 12000 pairs (ideal-noise). The data is split into training dataset (8000), validation dataset (2000) and test dataset (2000). For different sizes of the hidden layer of the neural network 
(D_hidden = 1, 2, 3, 4, 5, 6, 7, 7, 8, 9, 9, 10, 12, 25, 50, 100, 200) 50 models with different initial parameters are generated. The models are trained for 100 epochs with a batched size of 80 (100 batches total) and a learning rate of $lr = 3*10^{-4}$. The MSE error function is used.

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import pickle
import importlib
import os
from sklearn.model_selection import train_test_split
import random 

import torch
from torch import nn
import torch.nn.functional as F
from torch import optim
from torch import nn
import torch.nn.functional as F
from torchsummary import summary

import nn_functions
importlib.reload(nn_functions)
import functions as func
importlib.reload(func)

<module 'functions' from '/home/VNIIA/dvbabukhin/Загрузки/paper_code/functions.py'>

### Upload data

In [3]:
cwd = os.getcwd()
new_dir = cwd + '/data_q_01_J_half_h'

#### Echo-evolution data

In [4]:
"""
data['data'].shape = (number of initial states, number of time points, noise-free(0) or noisy(1), number of spins)
"""

data_name = '/data_t_5'

with open(new_dir + data_name + '.pkl', 'rb') as f:
    train_data_loaded = pickle.load(f)
    
del train_data_loaded['parameters']['init state']
print(train_data_loaded['data'].shape)

(2400, 5, 2, 6)


In [5]:
# Convert qubit excitation to spin magnetization
X = train_data_loaded['data'][:,:,1,:]
X = 2*X - 1
y = train_data_loaded['data'][:,:,0,:]
y = 2*y - 1

# Upload time points
time_points = np.linspace(0, train_data_loaded['parameters']['total sim time'], train_data_loaded['parameters']['time points'])
tn = X.shape[1] # number of time points

# Expand data from different data points
X_new = []
for k in range(X.shape[1]):
    X_new += [*X[:,k,:]]
X_new = np.array(X_new)
X = X_new

y_new = []
for k in range(y.shape[1]):
    y_new += [*y[:,k,:]]
y_new = np.array(y_new)
y = y_new

# Divide data into train and test sets
X_train_val, X_test, y_train_val, y_test = train_test_split(X, y, test_size=0.1666)
X_train_val = torch.from_numpy(X_train_val)
X_test  = torch.from_numpy(X_test)
y_train_val = torch.from_numpy(y_train_val)
y_test  = torch.from_numpy(y_test)

# Divide training data into training and validation sets
valid_size = 0.2
num_train_val = len(X_train_val)
indices = list(range(num_train_val))
np.random.shuffle(indices)
split = int(np.floor(valid_size * num_train_val))
train_idx, valid_idx = indices[split:], indices[:split]

X_train = X_train_val[train_idx]    
y_train = y_train_val[train_idx]

X_valid = X_train_val[valid_idx]    
y_valid = y_train_val[valid_idx]
    
print("data shape: ", train_data_loaded['data'].shape)
print(X.shape, X_train.shape, X_valid.shape, X_test.shape)
print(y.shape, y_train.shape, y_valid.shape, y_test.shape)

k = 0

print("Noisy data vector t:    ", X_train[k])
print("Noise-free data vector t: ", y_train[k])

data shape:  (2400, 5, 2, 6)
(12000, 6) torch.Size([8000, 6]) torch.Size([2000, 6]) torch.Size([2000, 6])
(12000, 6) torch.Size([8000, 6]) torch.Size([2000, 6]) torch.Size([2000, 6])
Noisy data vector t:     tensor([ 0.3624, -0.2286, -0.1950, -0.0500, -0.2198, -0.0932],
       dtype=torch.float64)
Noise-free data vector t:  tensor([ 0.9134, -0.5824, -0.9238, -0.3568, -0.9490, -0.4934],
       dtype=torch.float64)


In [5]:
cwd = os.getcwd()
new_dir = cwd + '/data_q_01_J_half_h'
new_dir += '/loss_experiment'

if not os.path.isdir(new_dir):
    os.mkdir(new_dir)

In [6]:
file_name = 'train_data'
output = open(new_dir + '/{}.pkl'.format(file_name), 'wb')
train_data = {"X": X_train, "Y": y_train}
pickle.dump(train_data, output)
output.close()

file_name = 'valid_data'
output = open(new_dir + '/{}.pkl'.format(file_name), 'wb')
valid_data = {"X": X_valid, "Y": y_valid}
pickle.dump(valid_data, output)
output.close()

file_name = 'test_data'
output = open(new_dir + '/{}.pkl'.format(file_name), 'wb')
test_data = {"X": X_test, "Y": y_test}
pickle.dump(test_data, output)
output.close()

## Training neural networks

In [7]:
D_in = X_train[0].shape[0]
D_hidden_lst = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 12, 25, 50, 100, 200]
Num_models = 50
epochs = 100

lr_0 = 3e-4
criterion = nn.MSELoss()
N_batches = 80
batch_size = int(X_train.shape[0]/N_batches)
print(batch_size)

model_matrix = [ [nn_functions.generate_onelayer_model(D_in, D_hidden, out_f='tanh') for _ in range(Num_models)] for D_hidden in D_hidden_lst]
for models in model_matrix:
    for model in models:
        model.double()

100


In [8]:
train_losses_array = np.zeros((len(D_hidden_lst), Num_models, epochs))
valid_losses_array = np.zeros((len(D_hidden_lst), Num_models, epochs))

for k, d in enumerate(D_hidden_lst):
    
    print("Dhidden: ", d)
    
    models = model_matrix[k]
    optimizers = [optim.Adam(model.parameters(), lr=lr_0) for model in models]
    
    for epoch in range(epochs):
        
        # Shuffle training set
        num_train = len(X_train)
        indices = list(range(num_train))
        np.random.shuffle(indices)
        X_train = X_train[indices]
        y_train = y_train[indices]
        # Shuffle validation set
        num_valid = len(X_valid)
        indices = list(range(num_valid))
        np.random.shuffle(indices)
        X_valid = X_valid[indices]
        y_valid = y_valid[indices]

        train_losses = np.zeros(len(models))
            
        for i in range(N_batches):
            
            x, y = X_train[i*batch_size:(i+1)*batch_size, :], y_train[i*batch_size:(i+1)*batch_size, :]
            
            for l, (optimizer, model) in enumerate(zip(optimizers, models)):
                optimizer.zero_grad()
                y_ = model(x)
                loss = criterion(y, y_)
                train_losses[l] += loss.item()
                loss.backward()
                optimizer.step()    
        
        else:
            valid_losses = np.zeros(len(models))
            with torch.no_grad():
                for x, y in zip(X_valid, y_valid):
                    
                    for l, model in enumerate(models):
                        y_ = model(x)
                        loss = criterion(y, y_)
                        valid_losses[l] += loss.item()
        
        for l, _ in enumerate(models):
            train_losses_array[k][l][epoch] = train_losses[l]/len(X_train)
            valid_losses_array[k][l][epoch] = valid_losses[l]/len(X_valid) 
            
            
file_name = 'trained_model_matrix'
output = open(new_dir + '/{}.pkl'.format(file_name), 'wb')
pickle.dump(model_matrix, output)
output.close()

file_name = 'train_losses'
output = open(new_dir + '/{}.pkl'.format(file_name), 'wb')
pickle.dump(train_losses_array, output)
output.close()

file_name = 'valid_losses'
output = open(new_dir + '/{}.pkl'.format(file_name), 'wb')
pickle.dump(valid_losses_array, output)
output.close()

Dhidden:  1
Dhidden:  2
Dhidden:  3
Dhidden:  4
Dhidden:  5
Dhidden:  6
Dhidden:  7
Dhidden:  8
Dhidden:  9
Dhidden:  10
Dhidden:  12
Dhidden:  25
Dhidden:  50
Dhidden:  100
Dhidden:  200
